In [21]:
# Import Module

import tkinter as tk
import pyodbc
from tkinter import ttk  
from tkinter import messagebox
from tkinter.filedialog import asksaveasfilename
import csv
#import unicodecsv as csv
import sys
import codecs
import tkinter.font as font
# Information List
datas = []
LARGEFONT =("Verdana", 35)
SMALLFONT=("Verdana", 13)

  
  
# Create Object
#root = tk.Tk()

#root.title("HỆ THỐNG QUẢN LÝ GIẢNG VIÊN")   
# Set geometry
#root.geometry('768x500+350+250')
conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=MON-MON-PC;DATABASE=QuanLyGiangVien;Trusted_Connection=yes;')
curѕor = conn.cursor()
#Add menu
def donothing():
   x = 0
#menubar = tk.Menu(root)
#filemenu = tk.Menu(menubar, tearoff=0)
#filemenu.add_command(label="Quản lý Giảng viên", command=donothing)
#filemenu.add_command(label="Quản lý khoa", command=donothing)
#filemenu.add_command(label="Quản lý Quản lý môn học", command=donothing)
#filemenu.add_separator()
#filemenu.add_command(label="Thoát", command=root.destroy)
#menubar.add_cascade(label="Chức năng", menu=filemenu)
#root.config(menu=menubar)

class tkinterApp(tk.Tk):
     # Add Buttons, Label, ListBox

    # __init__ function for class tkinterApp
    def __init__(self, *args, **kwargs):
         
        # __init__ function for class Tk
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("HỆ THỐNG QUẢN LÝ GIẢNG VIÊN")
  
        # creating a container
        container = tk.Frame(self) 
        container.pack(side = "top", fill = "both", expand = True)
  
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
  
        # initializing frames to an empty array
        self.frames = {} 
  
        # iterating through a tuple consisting
        # of the different page layouts
        for F in (GiangVienPage, MonHocPage, GiangVienMonHocPage):
  
            frame = F(container, self)
  
            # initializing frame of that object from
            # GiangVienPage, MonHocPage, GiangVienMonHocPage respectively with
            # for loop
            self.frames[F] = frame
            frame.grid(row = 0, column = 0, sticky ="nsew")
  
        self.show_frame(GiangVienPage)
  
    # to display the current frame passed as
    # parameter
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
  
# first window frame GiangVienPage
  
class GiangVienPage(tk.Frame):
    
    def __init__(self, parent, controller):
        self.parent = parent
        
        tk.Frame.__init__(self, parent)
        MaGV = tk.StringVar()
        HoDem = tk.StringVar()
        Ten = tk.StringVar()
        SDT = tk.StringVar()
        CCCD = tk.StringVar()
        DiaChi = tk.StringVar()
        NgaySinh = tk.StringVar()  
        HocVi = tk.StringVar() 
        NghiHuu = tk.BooleanVar()
        GioiTinh = tk.BooleanVar()
        IDKhoa = tk.IntVar()
        TuKhoa = tk.StringVar()
        def show_item():  
            dsGiangVien.delete(0,tk.END)
            dsGiangVien.insert(tk.END, f'Mã GV - Họ tên - Khoa CQ - Ngày sinh - Điện thoại')
            dsGVFull = curѕor.execute("EXEC dbo.SP_LoadGiangVien ?, null", (TuKhoa.get()))
            for c in dsGVFull.fetchall():
                dsGiangVien.insert(tk.END, f'{c[0]} - {c[1]} - {c[2]} {c[3]} - {c[4]} - {c[5]} - {c[6]}')
        def remove_item():
            selection = dsGiangVien.curselection()
            if len(selection) > 0:
                item = dsGiangVien.get(selection[0])
                if item.split(' - ')[0] != 'Mã GV':
                    res = messagebox.askquestion('Xác nhận', 
                                                 'Bạn chắc chắn muốn xóa Giảng viên '+ item.split(' - ')[1] + ' - ' + item.split(' - ')[2] + '?')
                    if res == 'yes':
                        id = int(item.split(' - ')[0])
                        curѕor.execute("EXEC dbo.SP_XoaGiangVien ?", (id))
                        curѕor.commit()
                        show_item()
        
        def add_item():
            IDKhoa = ({i for i in lstTenKhoa if lstTenKhoa[i]==cboKhoa.get()}).pop()
            curѕor.execute("EXEC dbo.SP_CapNhatGiangVien ?,?,?,?,?,?,?,?,?,?,?", 
                           (MaGV.get(), HoDem.get(),Ten.get(),GioiTinh.get(),NgaySinh.get(),HocVi.get(),IDKhoa,CCCD.get(),SDT.get(), DiaChi.get(), NghiHuu.get()))
            curѕor.commit()
            show_item()

        def reset_item():
            MaGV.set('')
            HoDem.set('')
            Ten.set('')
            SDT.set('')
            CCCD.set('')
            DiaChi.set('')
            NgaySinh.set('') 
            HocVi.set('')
            NghiHuu.set(False)
            GioiTinh.set(False)
            IDKhoa.set(0)
            cboKhoa.set('')
            TuKhoa.set('')

        def loadCurrent(event):
            curGV = dsGiangVien.get(dsGiangVien.curselection()).split('-')
            gv = curѕor.execute("EXEC dbo.SP_LoadGiangVien ?, null", (curGV[1].strip())).fetchall()[0]
            MaGV.set(gv[1])
            HoDem.set(gv[2])
            Ten.set(gv[3])
            SDT.set(gv[6])
            CCCD.set(gv[10])
            DiaChi.set(gv[7])
            NgaySinh.set(gv[5]) 
            HocVi.set(gv[9])
            NghiHuu.set(gv[12])
            GioiTinh.set(gv[13])
            IDKhoa.set(gv[11])
            cboKhoa.set(gv[4])
        def ExportCSV():                
            try:
                name = asksaveasfilename()
                dsGVFull = curѕor.execute("EXEC dbo.SP_LoadGiangVien ?, null", (TuKhoa))
            
                #with open('name.csv', 'w', encoding='utf-16') as csvfile:
                    #create = csv.writer(csvfile,quoting=csv.QUOTE_ALL)
                    #for c in dsGVFull.fetchall():
                        #csvfile.write(f'{c[0]}, {c[1]}, {c[2]} {c[3]}, {c[4]}, {c[5]}, {c[6]}\n')
            
                file = open (name,"w",encoding='utf-16')
                create = csv.writer(file)
                for c in dsGVFull.fetchall():
                    file.write(f'{c[1]}, {c[2]} {c[3]}, {c[4]}, {c[5]}, {c[6]}\n')

                file.close()
                messagebox.showinfo("Information", f"Xuất file {names}.csv thành công")
            except Exception as e:
                messagebox.showinfo("Error!!!", str(e))
        btnMH = ttk.Button(self, text ="Quản lý môn học", 
        command = lambda : controller.show_frame(MonHocPage))
        btnMH.grid(row = 0, column = 0, padx = 10, pady = 10)
        
         
        btnGVMH = ttk.Button(self, text ="Quản lý Giảng viên - môn học",
        command = lambda : controller.show_frame(GiangVienMonHocPage))
        btnGVMH.grid(row = 0, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Mã Giảng viên', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 1, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = MaGV,width=50).grid(row = 1, column = 1, padx = 10, pady = 10,sticky=tk.W)
        tk.Checkbutton(self, text = "Chấm dứt hợp đồng",font=SMALLFONT, variable=NghiHuu, onvalue = 1, offvalue = 0).grid(row = 1, column = 2, padx = 10, pady = 10,sticky=tk.W)
        
        tk.Label(self, text = 'Họ đệm', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 2, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = HoDem,width=50).grid(row = 2, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Tên', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 2, column = 2, padx = 10, pady = 10)
        tk.Entry(self, textvariable = Ten,width=50).grid(row = 2, column = 3, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Ngày sinh', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 3, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = NgaySinh,width=50).grid(row = 3, column = 1, padx = 10, pady = 10)
        
        tk.Radiobutton(self, text = "Nam",font=SMALLFONT, variable=GioiTinh, value=0, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 3, column = 2, padx = 10, pady = 10)
        tk.Radiobutton(self, text = "Nữ",font=SMALLFONT, variable=GioiTinh, value=1, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 3, column = 3, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Học vị', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 4, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = HocVi,width=50).grid(row = 4, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'CCCD', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 4, column = 2, padx = 10, pady = 10)
        tk.Entry(self, textvariable = CCCD,width=50).grid(row = 4, column = 3, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Số điện thoại', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 5, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = SDT,width=50).grid(row = 5, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Khoa', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 5, column = 2, padx = 10, pady = 10)
        
        query="SELECT DISTINCT TenKhoa, Id FROM dbo.Khoa"
                
        lstKhoa =conn.execute(query)
        lstTenKhoa = {}
        for columnrow in lstKhoa.fetchall():
            lstTenKhoa[columnrow[1]] = columnrow[0]
        khoa =tk.StringVar() # string variable 
        dsKhoa = list(lstTenKhoa.values())
        cboKhoa = ttk.Combobox(self, values=dsKhoa, textvariable=khoa)
        cboKhoa.grid(row = 5, column = 3, padx = 10, pady = 10,sticky=tk.W+tk.E)
        cboKhoa.set(dsKhoa[0])
        TuKhoa.set('')
        tk.Label(self, text = 'Địa chỉ liên hệ', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =6, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = DiaChi).grid(row =6, column=1,columnspan = 3, padx = 10, pady = 10,sticky = tk.W+tk.E)
  
        tk.Label(self, text = 'Từ khóa (mã/ họ tên GV:)', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =7, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = TuKhoa).grid(row =7, column=1,columnspan = 2, padx = 10, pady = 10,sticky = tk.W+tk.E)
        tk.Button(self,text="Xem danh sách",font=SMALLFONT,command=show_item).grid(row = 7, column = 3, padx = 10, pady = 10)
    
    
        scroll_bar = tk.Scrollbar(self, orient=tk.VERTICAL)
        dsGiangVien = tk.Listbox(self, yscrollcommand=scroll_bar.set, height=12)
        scroll_bar.config(command=dsGiangVien.yview)
        scroll_bar.grid(row = 8, column = 3, padx = 10, pady = 10)
        dsGiangVien.grid(row = 8, column = 0, columnspan = 4, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsGiangVien.bind("<<ListboxSelect>>", loadCurrent)

        tk.Button(self,text="Thêm mới",font= SMALLFONT,command=reset_item).grid(row = 9, column = 0, padx = 10, pady = 10)
        tk.Button(self,text="Lưu",font=SMALLFONT,command=add_item).grid(row = 9, column = 1, padx = 10, pady = 10)
        tk.Button(self,text="Xóa",font=SMALLFONT,command=remove_item).grid(row = 9, column = 2, padx = 10, pady = 10)
        tk.Button(self,text="Xuất file csv",font=SMALLFONT,command=ExportCSV).grid(row = 9, column = 3, padx = 10, pady = 10)

        
# second window frame MonHocPage
class MonHocPage(tk.Frame):
     
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        MaMH = tk.StringVar()
        TenMH = tk.StringVar()
        TuKhoa = tk.StringVar()
        def show_item():  
            dsMH.delete(0,tk.END)
            dsMH.insert(tk.END, f'Id - Mã môn học - Tên môn học')
            dsMHFull = curѕor.execute("EXEC dbo.SP_LoadMonHoc ?", (TuKhoa.get()))
            for c in dsMHFull.fetchall():
                dsMH.insert(tk.END, f'{c[0]} - {c[1]} - {c[2]}')
        def remove_item():
            selection = dsMH.curselection()
            if len(selection) > 0:
                item = dsMH.get(selection[0])
                if item.split(' - ')[0] != 'Mã môn học':
                    res = messagebox.askquestion('Xác nhận', 
                                                 'Bạn chắc chắn muốn xóa môn học '+ item.split(' - ')[1] + ' - ' + item.split(' - ')[2] + '?')
                    if res == 'yes':
                        id = int(item.split(' - ')[0])
                        curѕor.execute("EXEC dbo.SP_XoaMonHoc ?", (id))
                        curѕor.commit()
                        show_item()
        
        def add_item():
            curѕor.execute("EXEC dbo.SP_CapNhatMonHoc ?,?", (MaMH.get(),TenMH.get()))
            curѕor.commit()
            show_item()

        def reset_item():
            MaMH.set('')
            TenMH.set('')
            
        def loadCurrent(event):
            curMH = dsMH.get(dsMH.curselection()).split('-')
            mh = curѕor.execute("EXEC dbo.SP_LoadMonHoc ?", (curMH[1].strip())).fetchall()[0]
            MaMH.set(mh[1])
            TenMH.set(mh[2])
            
        btnGV = tk.Button(self, text ="Quản lý Giảng viên",
                            command = lambda : controller.show_frame(GiangVienPage))
     
        btnGV.grid(row = 0, column = 0, padx = 10, pady = 10)
  
        btnGVMH = tk.Button(self, text ="Quản lý Giảng viên - môn học",
                            command = lambda : controller.show_frame(GiangVienMonHocPage))
     
        btnGVMH.grid(row = 0, column = 1, padx = 10, pady = 10)
  
        tk.Label(self, text = 'Mã môn học', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 1, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = MaMH,width=60).grid(row = 1, column = 1, padx = 10, pady = 10,sticky=tk.W)
        tk.Label(self, text = 'Tên môn học', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 1, column = 2, padx = 10, pady = 10)
        tk.Entry(self, textvariable = TenMH,width=60).grid(row = 1, column = 3, padx = 10, pady = 10,sticky=tk.W)
        
        tk.Label(self, text = 'Từ khóa (mã/ tên MH:)', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =2, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = TuKhoa).grid(row =2, column=1, columnspan=2,padx = 10, pady = 10,sticky = tk.W+tk.E)
        tk.Button(self,text="Xem danh sách",font=SMALLFONT,command=show_item).grid(row = 2, column = 3, padx = 10, pady = 10)
        
        scroll_barMH = tk.Scrollbar(self, orient=tk.VERTICAL)
        dsMH = tk.Listbox(self, yscrollcommand=scroll_barMH.set, height=12,width=50)
        scroll_barMH.config(command=dsMH.yview)
        scroll_barMH.grid(row = 3, column = 3, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsMH.grid(row = 3, column = 0, columnspan = 3, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsMH.bind("<<ListboxSelect>>", loadCurrent)
        
        tk.Button(self,text="Thêm mới",font= SMALLFONT,command=reset_item).grid(row = 5, column = 0, padx = 10, pady = 10,sticky = tk.W)
        tk.Button(self,text="Lưu",font=SMALLFONT,command=add_item).grid(row = 5, column = 1, padx = 10, pady = 10)
        tk.Button(self,text="Xóa",font=SMALLFONT,command=remove_item).grid(row = 5, column = 2, padx = 10, pady = 10, sticky = tk.W)
# third window frame GiangVienMonHocPage
class GiangVienMonHocPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)

        btnMH = tk.Button(self, text ="Quản lý môn học",
                            command = lambda : controller.show_frame(MonHocPage))
     
        btnMH.grid(row = 0, column = 0, padx = 10, pady = 10)
  
        btnGV = tk.Button(self, text ="Quản lý Giảng viên",
                            command = lambda : controller.show_frame(GiangVienPage))
     
        btnGV.grid(row = 0, column = 1, padx = 10, pady = 10)
        
        


# Execute Tkinter
root = tkinterApp()
root.mainloop()

In [13]:
#pip install unicodecsv